# Object Detection with YOLO

This notebook demonstrates how to perform object detection on remote sensing imagery using the YOLO (You Only Look Once) model with the `ultralytics` library in Python. YOLO is effective for detecting objects like buildings, vehicles, or other features in high-resolution imagery.

## Prerequisites
- Install required libraries: `rasterio`, `ultralytics`, `numpy`, `matplotlib`, `geopandas` (listed in `requirements.txt`).
- A high-resolution GeoTIFF file (e.g., `sample.tif`) and a shapefile with bounding box annotations (e.g., `annotations.shp`). Replace file paths with your own data.
- GPU recommended for faster inference.

## Learning Objectives
- Prepare raster data and annotations for YOLO training.
- Fine-tune a YOLO model for object detection.
- Predict and visualize detected objects.

In [ ]:
# Import required libraries
import rasterio
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
from ultralytics import YOLO
from pathlib import Path
import os

## Step 1: Prepare Data for YOLO

Convert the raster and shapefile annotations into YOLO-compatible format (image patches and text files with bounding box coordinates).

In [ ]:
# Define file paths
image_path = 'sample.tif'
shapefile_path = 'annotations.shp'
output_dir = 'yolo_dataset/'
patch_size = 640  # YOLO expects square images

# Create directories
Path(output_dir).mkdir(exist_ok=True)
Path(f'{output_dir}/images').mkdir(exist_ok=True)
Path(f'{output_dir}/labels').mkdir(exist_ok=True)

# Load raster and shapefile
with rasterio.open(image_path) as src:
    image = src.read().transpose(1, 2, 0)  # Shape: (height, width, bands)
    transform = src.transform
    crs = src.crs
    profile = src.profile

gdf = gpd.read_file(shapefile_path)
if gdf.crs != crs:
    gdf = gdf.to_crs(crs)

# Extract patches and annotations
height, width = image.shape[:2]
for i in range(0, height, patch_size):
    for j in range(0, width, patch_size):
        # Extract patch
        patch = image[i:i+patch_size, j:j+patch_size]
        if patch.shape[:2] != (patch_size, patch_size):
            continue
        
        # Save patch as image
        patch_name = f'patch_{i}_{j}.jpg'
        plt.imsave(f'{output_dir}/images/{patch_name}', patch / patch.max() if patch.max() > 0 else patch)
        
        # Get patch bounds
        patch_bounds = rasterio.windows.from_bounds(
            *rasterio.transform.xy(transform, i, j, rows=[i, i+patch_size], cols=[j, j+patch_size]),
            transform=transform
        ).bounds
        
        # Filter annotations within patch bounds
        patch_gdf = gdf[gdf.geometry.intersects(gpd.GeoSeries.from_bounds(patch_bounds).geometry[0])]
        
        # Convert to YOLO format (class x_center y_center width height)
        with open(f'{output_dir}/labels/{patch_name.replace(".jpg", ".txt")}', 'w') as f:
            for _, row in patch_gdf.iterrows():
                class_id = row['class']  # Assumes 'class' column
                bounds = row.geometry.bounds
                x_min, y_min, x_max, y_max = bounds
                x_center = ((x_min + x_max) / 2 - patch_bounds[0]) / (patch_bounds[2] - patch_bounds[0])
                y_center = (1 - ((y_min + y_max) / 2 - patch_bounds[1]) / (patch_bounds[3] - patch_bounds[1]))
                width = (x_max - x_min) / (patch_bounds[2] - patch_bounds[0])
                height = (y_max - y_min) / (patch_bounds[3] - patch_bounds[1])
                f.write(f'{class_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}\n')

print(f'Dataset prepared in: {output_dir}')

## Step 2: Configure YOLO Dataset

Create a YAML configuration file for the YOLO dataset.

In [ ]:
# Create dataset YAML file
yaml_content = f"""
train: {output_dir}/images
val: {output_dir}/images
nc: {gdf['class'].nunique()}  # Number of classes
names: {list(gdf['class'].unique())}  # Class names
"""

with open(f'{output_dir}/dataset.yaml', 'w') as f:
    f.write(yaml_content)

print('Dataset YAML file created')

## Step 3: Train YOLO Model

Fine-tune a pretrained YOLO model on the prepared dataset.

In [ ]:
# Initialize YOLO model
model = YOLO('yolov8n.pt')  # Load pretrained YOLOv8 nano model

# Train model
model.train(
    data=f'{output_dir}/dataset.yaml',
    epochs=10,
    imgsz=patch_size,
    batch=8,
    device=0 if torch.cuda.is_available() else 'cpu'
)

## Step 4: Predict and Visualize Detections

Perform object detection on the raster and visualize results.

In [ ]:
# Load trained model (adjust path to trained weights)
model = YOLO('runs/detect/train/weights/best.pt')  # Update path after training

# Initialize output array for visualization
with rasterio.open(image_path) as src:
    image = src.read().transpose(1, 2, 0)
    height, width = image.shape[:2]

# Predict on patches
results = []
for i in range(0, height, patch_size):
    for j in range(0, width, patch_size):
        patch = image[i:i+patch_size, j:j+patch_size]
        if patch.shape[:2] != (patch_size, patch_size):
            continue
        result = model.predict(patch, save=False)
        results.append((result, i, j))

# Visualize detections
plt.figure(figsize=(10, 10))
plt.imshow(image / image.max() if image.max() > 0 else image)
for result, i, j in results:
    for box in result.boxes:
        x_min, y_min, x_max, y_max = box.xyxy[0].cpu().numpy()
        x_min, y_min, x_max, y_max = x_min + j, y_min + i, x_max + j, y_max + i
        plt.gca().add_patch(plt.Rectangle((x_min, y_min), x_max-x_min, y_max-y_min, edgecolor='red', facecolor='none', lw=2))
plt.title('YOLO Object Detection Results')
plt.xlabel('Column')
plt.ylabel('Row')
plt.show()

## Step 5: Save Detection Results

Save detected bounding boxes as a shapefile.

In [ ]:
# Create GeoDataFrame for detections
boxes = []
for result, i, j in results:
    for box in result.boxes:
        x_min, y_min, x_max, y_max = box.xyxy[0].cpu().numpy()
        x_min, y_min, x_max, y_max = x_min + j, y_min + i, x_max + j, y_max + i
        x_min_geo, y_max_geo = rasterio.transform.xy(transform, y_min, x_min)
        x_max_geo, y_min_geo = rasterio.transform.xy(transform, y_max, x_max)
        boxes.append({
            'geometry': gpd.GeoSeries.from_bounds((x_min_geo, y_min_geo, x_max_geo, y_max_geo)).geometry[0],
            'class': int(box.cls.cpu().numpy())
        })

gdf_detections = gpd.GeoDataFrame(boxes, crs=crs)
gdf_detections.to_file('yolo_detections.shp')

print('Detections saved to: yolo_detections.shp')

## Next Steps

- Replace `sample.tif` and `annotations.shp` with your own image and bounding box annotations.
- Adjust patch size, number of epochs, or use a different YOLO model (e.g., `yolov8m.pt`).
- Add evaluation metrics (e.g., mAP) for model performance.
- This is the final notebook in the series. Review previous notebooks for further analysis or explore advanced topics like multi-scale object detection.

## Notes
- Ensure the shapefile contains bounding box geometries with a 'class' column.
- YOLO requires images in RGB format; convert multi-band rasters if needed.
- See `docs/installation.md` for troubleshooting library installation.